In [6]:
!python -m pip install multiple-hypothesis-testing

In [7]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive/')
    # import glob; print(glob.glob('/content/gdrive/Othercomputers/My Laptop/projects/RUNI/Thesis/*'))
    import sys
    sys.path.append('/content/gdrive/Othercomputers/My Laptop/projects/RUNI/Thesis')
except:
    pass

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [8]:
from Synthetic_Data_Generators import Multi_Class_Normal_Population as Data_Generator
from Synthetic_Data_Generators import signal_2_noise_roc
from Higher_Criticism import Higher_Criticism
higher_criticism = Higher_Criticism(use_import=False, correct_symetric_baseline_p_values=False)

In [9]:
import scipy
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import functools
from sklearn import metrics

plt.rcParams['font.family'] = 'monospace'


In [10]:
monte_carlo=10000
N_range = [1000, 2000, 3000, 5000, 6000, 7000, 8000, 10000, 20000, 30000]
r_range = [0.1]  # , 0.3, 0.7, 1.0, 1.3, 1.7, 2.0]
beta_range = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.9, 1.0]
collect_results = {}
for ind_N, N in enumerate(N_range):
    print(f'Working on sample size: {N}')
    for ind_beta, beta in tqdm(list(enumerate(beta_range))):
        for ind_r, r in enumerate(r_range):
            signal_generator = Data_Generator(**Data_Generator.params_from_N_r_beta(N=N, r=r, beta=beta))
            hc_monte_carlo = Higher_Criticism.monte_carlo_statistics_HC(monte_carlo=monte_carlo, data_generator=signal_generator)
            for key, auc in hc_monte_carlo.items():
                if key not in collect_results:
                    collect_results[key] = np.empty(shape=(len(N_range),len(r_range),len(beta_range)), dtype=np.float32)
                collect_results[key][ind_N, ind_r, ind_beta] = auc

for ind_r, r in enumerate(r_range):
    for ind_beta, beta in enumerate(beta_range):
        fig, ax = plt.subplots(1, 1, figsize=(6, 6))
        for key in collect_results:
            auc = collect_results[key][:, ind_r, ind_beta].reshape(-1)
            ax.plot(N_range, auc, label=key)
        ax.set_title(f'AUC values as function of number of samples using {monte_carlo} monte carlo runs.\n' + f'r={r:.2f} beta={beta:.2f}')
        plt.legend()
        plt.show()


Working on sample size: 1000


100%|██████████| 9/9 [07:36<00:00, 50.70s/it]


Working on sample size: 2000


 33%|███▎      | 3/9 [04:50<09:40, 96.68s/it]


KeyboardInterrupt: ignored

In [ ]:
monte_carlo=10000
N_range = [1000, 2000, 3000, 5000, 6000, 7000, 8000, 10000, 20000, 30000]
r_range = [0.1]  # , 0.3, 0.7, 1.0, 1.3, 1.7, 2.0]
beta_range = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.9, 1.0]
collect_results = {}
for ind_N, N in enumerate(N_range):
    print(f'Working on sample size: {N}')
    noise_generator = Data_Generator(**Data_Generator.params_pure_noise(N = N))
    noise_monte_carlo = higher_criticism.monte_carlo_statistics(monte_carlo=monte_carlo, data_generator=noise_generator, disable_tqdm=True)
    for ind_r, r in tqdm(list(enumerate(r_range))):
        for ind_beta, beta in enumerate(beta_range):
            signal_generator = Data_Generator(**Data_Generator.params_from_N_r_beta(N=N, r=r, beta=beta))
            singal_monte_carlo = higher_criticism.monte_carlo_statistics(monte_carlo=monte_carlo, data_generator=signal_generator, disable_tqdm=True)
            for key, noise_values in noise_monte_carlo.items():
                if key in ['nums_rejected', 'first_drawdown']:
                    continue
                if key not in collect_results:
                    collect_results[key] = np.empty(shape=(len(N_range),len(r_range),len(beta_range)), dtype=np.float32)
                signal_values = singal_monte_carlo[key]
                auc, _, _ = signal_2_noise_roc(signal_values=signal_values, noise_values=noise_values)
                collect_results[key][ind_N, ind_r, ind_beta] = auc



In [ ]:
for ind_r, r in enumerate(r_range):
    for ind_beta, beta in enumerate(beta_range):
        fig, ax = plt.subplots(1, 1, figsize=(6, 6))
        for key in collect_results:
            auc = collect_results[key][:, ind_r, ind_beta].reshape(-1)
            ax.plot(N_range, auc, label=key)
        ax.set_title(f'AUC values as function of number of samples using {monte_carlo} monte carlo runs.\n' + f'r={r:.2f} beta={beta:.2f}')
        plt.legend()
        plt.show()
